In [75]:
import requests
import json
import pandas as pd
from config import api_key

In [88]:
filepath = "Resources/Item_Codes.csv"
item_codes_df = pd.read_csv(filepath)
item_codes_df

,SUB CATEGORY CODE,ITEM CODE,DESCRIPTION
0,ALCBEVG,ALCBEVG,Alcoholic beverages
1,APPAREL,APPAREL,Apparel and services
2,APPAREL,BOYS,"Apparel, Boys, 2 to 15"
3,APPAREL,FOOTWEAR,Footwear
4,APPAREL,GIRLS,"Apparel, Girls, 2 to 15"
...,...,...,...
149,TRANS,USEDCARS,"Vehicle purchases: Cars and trucks, used"
150,TRANS,VEHFINCH,Vehicle finance charges
151,TRANS,VEHOTHXP,Other vehicle expenses
152,TRANS,VEHPURCH,Vehicle purchases (net outlay)


In [101]:
# A single BLS API query can take up to 50 series ID at once. Since there are 153 item codes, 
# we created 4 nested lists of series IDs to request expenditure all item codes in 4 separate API queries
seriesids = [[],[],[],[]]

# Row counts tracks the limit of 50 elements per list. Set count tracks which nest list we're appending to.
setcount = 0
rowcount = 0

# Iterate through each row in the item_codes_df DataFrame
for index,row in item_codes_df.iterrows():
    # Move to the next nested series id list if the rowcount exceeds the API query limit of 50
    if rowcount == 50:
        setcount = setcount + 1
        rowcount = 0
    
    # Use the Item Code value in the DataFrame to modify the series ID and append the value to the list
    seriesids[setcount].append(f'CXU{row["ITEM CODE"]}LB0101M')
    
    # Iterate row count
    rowcount = rowcount + 1 
    
len(seriesids)

4

In [78]:
# Define header to post HTTP request
headers = {'Content-type': 'application/json'}

# Create a list to store the output of each API query
json_data = []

# Loop through the nested series ids to request up to 50 series IDs API per query
for i in range(len(seriesids))):
    data = json.dumps({"seriesid": seriesids[i],"startyear":"2010", "endyear":"2011","registrationkey":api_key})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data.append(json.loads(p.text))

In [109]:
# This cell is used to test accessing the elements stored in the bls json reponse
len(json_data[1]["Results"]['series'][22]['data'])

2

In [123]:
# Initialize set and row count with the same meaning as above
rowcount = 0
setcount = 0

# Initialize lists that will be used as columns in the new DataFrame 
item_year = []
item_value = []
item_cat = []
item_desc = []

for i in range(len(item_codes_df)):
    if rowcount == 50:
        setcount = setcount + 1
        rowcount = 0
    
    # Iterate through the number of years in the API query
    for j in range(len(json_data[1]["Results"]['series'][22]['data'])):
        
        # Attempts to return the year and value from the json API reponse
        try:
            item_year.append(json_data[setcount]["Results"]['series'][rowcount]['data'][j]['year'])
            item_value.append(json_data[setcount]["Results"]['series'][rowcount]['data'][j]['value'])
        
        # If fails, add a null value to the list (all lists need to be the same size when creating the DataFrame)
        except:
            item_year.append('')
            item_value.append('')
            print(f"Item {i} doesn't have data")
        
        # Create list for item category and item description for the new DataFrame
        item_cat.append(item_codes_df.iloc[i,0])
        item_desc.append(item_codes_df.iloc[i,2])
        
    # Iterate row count
    rowcount = rowcount + 1
 
# Create new DataFrame with one column for year
item_data_df = pd.DataFrame({"Category": item_cat,
                             "Item":item_desc,
                             "Year": item_year,
                             "Value": item_value
})

item_data_df

Item 20 doesn't have data
Item 20 doesn't have data
Item 22 doesn't have data
Item 22 doesn't have data
Item 24 doesn't have data
Item 24 doesn't have data
Item 57 doesn't have data
Item 57 doesn't have data
Item 80 doesn't have data
Item 80 doesn't have data
Item 91 doesn't have data
Item 91 doesn't have data
Item 114 doesn't have data
Item 114 doesn't have data
Item 134 doesn't have data
Item 134 doesn't have data
Item 135 doesn't have data
Item 135 doesn't have data
Item 136 doesn't have data
Item 136 doesn't have data
Item 137 doesn't have data
Item 137 doesn't have data
Item 138 doesn't have data
Item 138 doesn't have data


,Category,Item,Year,Value
0,ALCBEVG,Alcoholic beverages,2011,456
1,ALCBEVG,Alcoholic beverages,2010,412
2,APPAREL,Apparel and services,2011,1740
3,APPAREL,Apparel and services,2010,1700
4,APPAREL,"Apparel, Boys, 2 to 15",2011,80
...,...,...,...,...
303,TRANS,Other vehicle expenses,2010,2464
304,TRANS,Vehicle purchases (net outlay),2011,2669
305,TRANS,Vehicle purchases (net outlay),2010,2588
306,TRANS,"Vehicle. rent., leas., licen., oth. charges",2011,433
